<a href="https://colab.research.google.com/github/afroman32/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

We will focus specifically on Shakespeare's Sonnets in order to improve our model's ability to learn from the data.

In [65]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional
from tensorflow.keras.layers import LSTM

%matplotlib inline

# a custom data prep class that we'll be using 
from data_cleaning_toolkit_class import data_cleaning_toolkit

### Use request to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) in order to learn how to download the Shakespeare Sonnets from the Gutenberg website. 

**Protip:** Do not over think it.

In [66]:
# download all of Shakespears Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use request and the url to download all of the sonnets - save the result to `r`

r = requests.get(url_shakespeare_sonnets)


In [67]:
# move the downloaded text out of the request object - save the result to `raw_text_data`
# hint: take at look at the attributes of `r`
raw_text_data = r.text
raw_text_data

'\ufeffThe Project Gutenberg EBook of Shakespeare\'s Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Shakespeare\'s Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nPosting Date: April 7, 2014 [EBook #1041]\r\nRelease Date: September, 1997\r\nLast Updated: March 10, 2010\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE\'S SONNETS ***\r\n\r\n\r\n\r\n\r\nProduced by Joseph S. Miller and Embry-Riddle Aeronautical\r\nUniversity Library. HTML version by Al Haines.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\n  I\r\n\r\n  From fairest creatures we desire increase,\r\n  That thereby beauty\'s rose might never die,\r\n  But as the r

In [68]:
# check the data type of `raw_text_data`
type(raw_text_data)

str

### Data Cleaning

In [69]:
# as usual, we are tasked with cleaning up messy data
# Question: Do you see any characters that we could use to split up the text?
raw_text_data[:3000]

"\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Shakespeare's Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nPosting Date: April 7, 2014 [EBook #1041]\r\nRelease Date: September, 1997\r\nLast Updated: March 10, 2010\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***\r\n\r\n\r\n\r\n\r\nProduced by Joseph S. Miller and Embry-Riddle Aeronautical\r\nUniversity Library. HTML version by Al Haines.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\n  I\r\n\r\n  From fairest creatures we desire increase,\r\n  That thereby beauty's rose might never die,\r\n  But as the riper

In [70]:
# split the text into lines and save the result to `split_data`
split_data = raw_text_data.split('\r\n')
# raw_text_data

In [71]:
# we need to drop all the boiler plate text (i.e. titles and descriptions) as well as white spaces
# so that we are left with only the sonnets themselves 
split_data[:20] 

["\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare",
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '',
 "Title: Shakespeare's Sonnets",
 '',
 'Author: William Shakespeare',
 '',
 'Posting Date: April 7, 2014 [EBook #1041]',
 'Release Date: September, 1997',
 'Last Updated: March 10, 2010',
 '',
 'Language: English',
 '',
 '',
 "*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***"]

**Use list index slicing in order to remove the titles and descriptions so we are only left with the sonnets.**


In [72]:
# sonnets exists between these indicies 
# titles and descriptions exist outside of these indicies

# use index slicing to isolate the sonnet lines - save the result to `sonnets`

sonnets = split_data[36:]
sonnets

['THE SONNETS',
 '',
 'by William Shakespeare',
 '',
 '',
 '',
 '',
 '  I',
 '',
 '  From fairest creatures we desire increase,',
 "  That thereby beauty's rose might never die,",
 '  But as the riper should by time decease,',
 '  His tender heir might bear his memory:',
 '  But thou, contracted to thine own bright eyes,',
 "  Feed'st thy light's flame with self-substantial fuel,",
 '  Making a famine where abundance lies,',
 '  Thy self thy foe, to thy sweet self too cruel:',
 "  Thou that art now the world's fresh ornament,",
 '  And only herald to the gaudy spring,',
 '  Within thine own bud buriest thy content,',
 "  And tender churl mak'st waste in niggarding:",
 '    Pity the world, or else this glutton be,',
 "    To eat the world's due, by the grave and thee.",
 '',
 '  II',
 '',
 '  When forty winters shall besiege thy brow,',
 "  And dig deep trenches in thy beauty's field,",
 "  Thy youth's proud livery so gazed on now,",
 "  Will be a tatter'd weed of small worth held:",
 '

In [73]:
# notice how all non-sonnet lines have far less characters than the actual sonnet lines?
# well, let's use that observation to filter out all the non-sonnet lines
sonnets[200:240]

["  And sable curls, all silvered o'er with white;",
 '  When lofty trees I see barren of leaves,',
 '  Which erst from heat did canopy the herd,',
 "  And summer's green all girded up in sheaves,",
 '  Borne on the bier with white and bristly beard,',
 '  Then of thy beauty do I question make,',
 '  That thou among the wastes of time must go,',
 '  Since sweets and beauties do themselves forsake',
 '  And die as fast as they see others grow;',
 "    And nothing 'gainst Time's scythe can make defence",
 '    Save breed, to brave him when he takes thee hence.',
 '',
 '  XIII',
 '',
 '  O! that you were your self; but, love you are',
 '  No longer yours, than you your self here live:',
 '  Against this coming end you should prepare,',
 '  And your sweet semblance to some other give:',
 '  So should that beauty which you hold in lease',
 '  Find no determination; then you were',
 "  Yourself again, after yourself's decease,",
 '  When your sweet issue your sweet form should bear.',
 '  Wh

In [74]:
# any string with less than n_chars characters will be filtered out - save results to `filtered_sonnets`

filtered_sonnets = [line for line in sonnets if len(line)>=15]
filtered_sonnets

['by William Shakespeare',
 '  From fairest creatures we desire increase,',
 "  That thereby beauty's rose might never die,",
 '  But as the riper should by time decease,',
 '  His tender heir might bear his memory:',
 '  But thou, contracted to thine own bright eyes,',
 "  Feed'st thy light's flame with self-substantial fuel,",
 '  Making a famine where abundance lies,',
 '  Thy self thy foe, to thy sweet self too cruel:',
 "  Thou that art now the world's fresh ornament,",
 '  And only herald to the gaudy spring,',
 '  Within thine own bud buriest thy content,',
 "  And tender churl mak'st waste in niggarding:",
 '    Pity the world, or else this glutton be,',
 "    To eat the world's due, by the grave and thee.",
 '  When forty winters shall besiege thy brow,',
 "  And dig deep trenches in thy beauty's field,",
 "  Thy youth's proud livery so gazed on now,",
 "  Will be a tatter'd weed of small worth held:",
 '  Then being asked, where all thy beauty lies,',
 '  Where all the treasu

In [75]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text
filtered_sonnets

['by William Shakespeare',
 '  From fairest creatures we desire increase,',
 "  That thereby beauty's rose might never die,",
 '  But as the riper should by time decease,',
 '  His tender heir might bear his memory:',
 '  But thou, contracted to thine own bright eyes,',
 "  Feed'st thy light's flame with self-substantial fuel,",
 '  Making a famine where abundance lies,',
 '  Thy self thy foe, to thy sweet self too cruel:',
 "  Thou that art now the world's fresh ornament,",
 '  And only herald to the gaudy spring,',
 '  Within thine own bud buriest thy content,',
 "  And tender churl mak'st waste in niggarding:",
 '    Pity the world, or else this glutton be,',
 "    To eat the world's due, by the grave and thee.",
 '  When forty winters shall besiege thy brow,',
 "  And dig deep trenches in thy beauty's field,",
 "  Thy youth's proud livery so gazed on now,",
 "  Will be a tatter'd weed of small worth held:",
 '  Then being asked, where all thy beauty lies,',
 '  Where all the treasu

### Use custom data cleaning tool 

Use one of the methods in `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [76]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`
dctk = data_cleaning_toolkit()

In [77]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`
sonnet_map = map(dctk.clean_data, filtered_sonnets)
clean_sonnets = list(sonnet_map)

In [78]:
# much better!
clean_sonnets

['by william shakespeare',
 'from fairest creatures we desire increase',
 'that thereby beautys rose might never die',
 'but as the riper should by time decease',
 'his tender heir might bear his memory',
 'but thou contracted to thine own bright eyes',
 'feedst thy lights flame with selfsubstantial fuel',
 'making a famine where abundance lies',
 'thy self thy foe to thy sweet self too cruel',
 'thou that art now the worlds fresh ornament',
 'and only herald to the gaudy spring',
 'within thine own bud buriest thy content',
 'and tender churl makst waste in niggarding',
 'pity the world or else this glutton be',
 'to eat the worlds due by the grave and thee',
 'when forty winters shall besiege thy brow',
 'and dig deep trenches in thy beautys field',
 'thy youths proud livery so gazed on now',
 'will be a tatterd weed of small worth held',
 'then being asked where all thy beauty lies',
 'where all the treasure of thy lusty days',
 'to say within thine own deep sunken eyes',
 'were an 

### Use your data tool to create character sequences 

We'll need the `create_char_sequenes` method for this task. However this method requires a parameter call `maxlen` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

In order to answer that question, let's do some statistics! 

In [79]:
def calc_stats(corpus):
    """
    Calculates statisics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnets line length - save the results to `doc_lens` 

    # use numpy to calcualte and return the mean, median, std, max, min of the doc lens - all in one line of code

    doc_lens = [len(line) for line in corpus]
    return (np.mean(doc_lens), np.median(doc_lens), np.std(doc_lens), np.max(doc_lens), np.min(doc_lens))

In [80]:
# sonnet line length statistics 
mean ,med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_ 

(43.11283728536386, 41.0, 8.43708031049097, 72, 14)

In [114]:
# using the results of the sonnet line length statistics, use your judgement and select a value for maxlen
# use .create_char_sequences() to create sequences
maxlen = 42
dctk.create_char_sequences(clean_sonnets, maxlen = maxlen )

sequences:  21572


Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. Let's call them in the cells below.

In [115]:
# number of input features for our LSTM model
dctk.n_features

27

In [116]:
dctk.unique_chars

['r',
 'h',
 'n',
 'c',
 'k',
 's',
 'q',
 'z',
 'o',
 'e',
 'm',
 't',
 'g',
 'd',
 'u',
 'i',
 'p',
 ' ',
 'b',
 'x',
 'a',
 'j',
 'v',
 'l',
 'w',
 'f',
 'y']

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e. **dctk.unique_chars**) and the `number of model input features` (i.e. **dctk.n_features**) the same?

**Hint:** The model that we will shortly be building here is very similar to the text generation model that we built in the guided project.

**Answer 1:**

They are both the letters of the alphabet and a whitespace character. Because the model takes in words as input and is generating words back.


**Question 2:**

Take a look at the print out of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

So that our generated text can have spaces in it.

----

### Use our data tool to create X and Y splits

You'll need the `create_X_and_Y` method for this task. 

In [117]:
# TODO: provide a walk through of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()

![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [118]:
# notice that our input matrix isn't actually a matrix - it's a rank 3 tensor
X.shape

(21572, 42, 27)

In $X$.shape we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [119]:
# first index returns a signle sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Notice that each sequence (i.e. $X[i]$ where $i$ is some index value) is `maxlen` long and has `dctk.n_features` number of features. Let's try to better understand this shape. 

In [120]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(42, 27)

**Each row corresponds to a character vector** and there are `maxlen` number of character vectors. 

**Each column corresponds to a unique character** and there are `dctk.n_features` number of features. 


In [121]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False])

Notice that there is a single `TRUE` value and all the rest of the values are `FALSE`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will necessarily be a single `TRUE` value and the rest will be `FALSE`. 

Let's say that `TRUE` appears in the $ith$ index; by  $ith$ index we simply mean some index in the general case. How can we find out which character that actually corresponds to?

To answer this question, we need to use the character-to-integer look up dictionaries. 

In [122]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that that character vector is endcoding for
dctk.int_char

{0: 'r',
 1: 'h',
 2: 'n',
 3: 'c',
 4: 'k',
 5: 's',
 6: 'q',
 7: 'z',
 8: 'o',
 9: 'e',
 10: 'm',
 11: 't',
 12: 'g',
 13: 'd',
 14: 'u',
 15: 'i',
 16: 'p',
 17: ' ',
 18: 'b',
 19: 'x',
 20: 'a',
 21: 'j',
 22: 'v',
 23: 'l',
 24: 'w',
 25: 'f',
 26: 'y'}

In [123]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

b
y
 
w
i
l
l
i
a
m
 
s
h
a
k
e
s
p
e
a
r
e
 
f
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
Sequence length: 42


## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape print out of `X.shape` to a fellow classmate?


**Answer 1:**

The first number represents the number of sequences we have, the second number represents how many characters are in each sequence which is set by maxlen, the last is the number of possibe values, which in this case is one for each letter of the alphabet and one space.

----


### Build a Text Generation model

Now that we have prepped our data (and understood that process) let's finally build out our character generation model, similar to what we did in the guided project.

In [124]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array
    """
    # convert preds to array 
    preds = np.asarray(preds).astype('float64')
    # scale values 
    preds = np.log(preds) / temperature
    # exponentiate values
    exp_preds = np.exp(preds)
    # this equation should look familar to you (hint: it's an activation function)
    preds = exp_preds / np.sum(exp_preds)
    # Draw samples from a multinomial distribution
    probas = np.random.multinomial(1, preds, 1)
    # return the index that corresponds to the max probability 
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    """"
    Function invoked at end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunce into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next 40 chars should be that follow the seed string
    for i in range(40):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [125]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)

In [126]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load in (the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [137]:
# build text generation model layer by layer 
# fit model

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, 27)))
model.add(Dense(27, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y,
          batch_size=32,
          epochs=200,
          callbacks=[print_callback])

Epoch 1/200
675/675 [==============================] - 4s 5ms/step - loss: 2.6202

----- Generating text after Epoch: 0
----- Generating with seed: "e subscribes since spite of him ill live i"
e subscribes since spite of him ill live itg on ninag io iha dothxuer bvefere pium
Epoch 2/200
675/675 [==============================] - 3s 5ms/step - loss: 2.2413

----- Generating text after Epoch: 1
----- Generating with seed: " that is that which it contains and that i"
 that is that which it contains and that illede shocr yfokibe thou ffere ty home m
Epoch 3/200
675/675 [==============================] - 3s 5ms/step - loss: 2.1096

----- Generating text after Epoch: 2
----- Generating with seed: " slander as i think proceeds thine eyes i "
 slander as i think proceeds thine eyes i ppaak nw thet oy ah pareispee tai hey ki
Epoch 4/200
675/675 [==============================] - 3s 5ms/step - loss: 2.0170

----- Generating text after Epoch: 3
----- Generating with seed: " days long since before

In [138]:
# save trained model to file 
model.save("makoa_text_gen_model.h5")

### Let's play with our trained model 

Now that we have a trained model that, though far from perfect, is able to generate actual English words, we can take a look at the predictions to continue to learn more about how a text generation model works. 

We can also take this as an opportunity to unpack the `def on_epoch_end` function to better understand how it works. 

In [139]:
# this is our joinned clean sonnet data
text

'by william shakespeare from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy foe to thy sweet self too cruel thou that art now the worlds fresh ornament and only herald to the gaudy spring within thine own bud buriest thy content and tender churl makst waste in niggarding pity the world or else this glutton be to eat the worlds due by the grave and thee when forty winters shall besiege thy brow and dig deep trenches in thy beautys field thy youths proud livery so gazed on now will be a tatterd weed of small worth held then being asked where all thy beauty lies where all the treasure of thy lusty days to say within thine own deep sunken eyes were an alleating shame and thriftless praise how much more praise deservd thy beautys use if tho

In [140]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

68641

In [141]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e. input seqeunce into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

generated

'se accumulate bring me within the level of'

In [142]:
# this block of code let's us know what the seed string is 
# i.e. the input seqeunce into the model
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

----- Generating with seed: "se accumulate bring me within the level of"
se accumulate bring me within the level of

In [143]:
# use model to predict what the next 40 chars should be that follow the seed string
for i in range(40):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [144]:
# this is the seed string
generated

'se accumulate bring me within the level of'

In [145]:
# these are the 40 chars that the model thinks should come after the seed stirng
sentence

'of dairters of einst dight by surmors of y'

In [146]:
# how put it all together
generated + sentence

'se accumulate bring me within the level ofof dairters of einst dight by surmors of y'

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN